In [38]:
import re
from ufal.udpipe import Model, Pipeline, ProcessingError
import pandas as pd 
from conllu import parse

model = Model.load('G:/Softwares/urdu-udtb-ud-2.3-181115.udpipe')
pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
error = ProcessingError()

In [39]:
from __future__ import unicode_literals
# -*- coding: utf-8 -*-
STOP_WORDS = """
 ابھی از 
 اس استعمال اسی اسے البتہ الف ان اندر اور اوپر اکثر اگر
 اگرچہ اگلے ایسا ایسی ایسے اے بار بارے باوجود باہر بظاہر بعض بغیر بلکہ بن
 بھر بھریں بھی بہت بے تا تاکہ تب تحت تر تمام
 تو تک جب سوال 
 جبکہ جو حالانکہ حالاں خلاف
 خود سے میں چنانچہ دیر ذریعے تھا هے ایک آپ
 سا ساتھ سامنے سب تھی تھے
 سکا سکتا سکتے سی سے شان شاید صرف صورت ضرورت ضروری طرح طرف طور علاوہ عین غیر 
 لہذا لیکن لیں لیے لے مجھ مجھے مزید مقابلے مل مکمل مگر 
 نا نہ نہیں نیچے واقعی والا والوں والی والے وجہ وغیرہ وہ وہاں وہی وہیں وی ویسے پاس
 پایا پر پوری پھر پیچھے چونکہ چکی
 ڈالے کئے کافی کبھی کسی کم 
 کوئی کچھ کہ کہا کہہ کہیں کہے کیونکہ کیے کے گئی
 گئے گا گویا گی گے ہاں ہر ہمیشہ ہو ہوئی ہوئیں ہوئے ہوا ہوتا
 ہوتی ہوتیں ہوتے ہونا ہونگے ہونی ہونے ہوں ہی ہیں ہے یا یات یعنی یقینا یہ یہاں یہی یہیں ھیں
  حضرت ہوگی رض
 کا کو کی نے ہوتو پہلے کر
 اپنا حتى كہ ميں لئیے  
 بعد ارادہ افاقہ آخر
""".split()

In [40]:
from gensim.models import word2vec
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('urduvec_140M_100K_300d.bin', binary=True)

## To get row count only

In [41]:
import pyodbc
from gensim.models import Word2Vec
import sys
import time
start = time.time()

conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=G:\Softwares\EQWH-V6.4-190407\EasyHadees - Copy.mdb;')
cursor = conn.cursor()
targetcount = 100
cursor.execute("select count(*) from ahadith where urdutext is not null and urdutext like '%وضو%'")

count = cursor.fetchone()[0]
print(count)
cursor.close()
conn.close()

6709


In [77]:
import pyodbc
from gensim.models import Word2Vec
import sys
import time
import numpy as np

no_of_docs_cache = {}
idf_cache = {}
term_freq_doc_cache = {}

conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=G:\Softwares\EQWH-V6.4-190407\EasyHadees - Copy.mdb;')
cursor = conn.cursor()
cursor.execute("select id, urdutext from ahadith where urdutext is not null and urdutext like '%وضو%'")

# and urdutext like '%وضو%'

Y = []
corpus = []
lens_of_d= []
for row in cursor.fetchall():
    hadith = row[1]#.replace('صلی اللہ علیہ وآلہ وسلم','')    
    corpus.append(hadith)
    lens_of_d.append(len(hadith.split()))
    Y.append(row[0])
cursor.close()
conn.close()

avg_docs_len = np.mean(lens_of_d, axis=0)

In [78]:
#from sklearn.feature_extraction.text import TfidfVectorizer as vectorizer
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(stop_words=STOP_WORDS).fit(corpus)
bag_of_words = vec.transform(corpus)
vocab = vec.vocabulary_.items()

# Ranking (okapi BM25)

![title](img/43e5c609557364f7836b6b2f4cd8ea41deb86a96.svg)

where

![title](img/c652b6871ce4872c8e924ff0f806bc8b06dc94ed.svg)

In [79]:
def term_freq_doc(myword,doc_index):
    if myword not in vec.vocabulary_:
        return 0
    
    qindex = vec.vocabulary_[myword]
    sm = None
    if doc_index in term_freq_doc_cache:
        sm = term_freq_doc_cache[doc_index]
    else:
        #sm = bag_of_words[doc_index].sum(axis=0)
        sm = bag_of_words[doc_index].toarray()[0]
        term_freq_doc_cache[doc_index] = sm
    freq = sm[qindex]
    return freq

In [80]:
#bag_of_words_arr = np.asarray(bag_of_words) # bag_of_words.toarray()

def no_of_docs(q):
    if q in no_of_docs_cache:
        return no_of_docs_cache[q]
    
    if q not in vec.vocabulary_:
        return 0
    
    qindex = vec.vocabulary_[q]
    
    no_of_ds = 0
    
    for bw in bag_of_words:
        if bw[0][0, qindex] > 0:
            no_of_ds += 1
    
    no_of_docs_cache[q] = no_of_ds;
    return no_of_ds        

In [81]:
import math

def idf(q):
    if q in idf_cache:
        return idf_cache[q]
    
    nq = no_of_docs(q)
    ans = math.log10(  (len(Y)- nq + 0.5) / (nq+ 0.5)  )
    idf_cache[q] = ans;
    return ans

def bm25_score(qwords,doc_index):
    score = 0;
    for qw in qwords:
        #start = time.time()
        tfd = term_freq_doc(qw, doc_index)
        score += idf(qw) * ( (tfd * (1.2 + 1))/ (tfd + 1.2 * (1-0.75 + 0.75 * (lens_of_d[doc_index] / avg_docs_len))) + 1.0)
        #end = time.time()
        #print(" time taken: "+str(end - start)+ ", for word: " + qw + ", for doc: " + str(doc_index))
    return score

In [82]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
def getanswer(question,strict=False,pagesize=10):
    question_arr = question.split()

    query_words = []
    
    for qs in question_arr:
        query_words.append(qs)
    
    if strict == False:
        for question_word in question_arr:
            alltop = model.most_similar(positive=question_word, topn=20)
            for key, val in [item for item in alltop if item[1] > 0.51 and item[0] in vec.vocabulary_]:
                query_words.append(key)
        
    doc_scores = []
    for i in range(0, len(Y)):
        start = time.time()
        score = bm25_score(query_words, i)
        end = time.time()
        #print(" time taken: "+str(end - start)+ ", for doc: " + str(i))
        doc_scores.append({'docid':i, 'score': score})

    doc_scores = sorted(doc_scores, key = lambda i: i['score'],reverse=True) 

    for i in range(0,pagesize):
        printmd(""+corpus[doc_scores[i]['docid']]+'')
        print()

In [86]:
#question = 'آپ کس طرح وضو کرتے تھے'
#question = 'وضو کس طرح ٹوٹتا ہے'
##question = 'وضو کس طرح کریں'
##question = 'وضو کا کیا طریقہ ہے'
#question = 'استنجا کیسے کریں'
getanswer(question, strict=False)


 ٢٢٨ : اعرج نے حضرت ابوہریرہ (رض) سے مرفوع روایت نقل کی کہ جناب رسول اللہ (صلی اللہ علیہ وآلہ وسلم) نے اسی طرح فرمایا۔ پس اس قول : ((لولا ان اشق علی امتی)) یعنی اگر میری امت پر گرانی نہ ہوتی تو میں ہر نماز کے لیے ان کو مسواک کا حکم دیتا حالانکہ آپ (صلی اللہ علیہ وآلہ وسلم) نے ان کو حکم نہیں دیا اور ان پر لازم بھی نہیں اور اس کے ختم کردینے میں جبکہ یہ ہر نماز کے لیے وضو کا بدل ہے تو اس میں اس بات کی دلیل ہے کہ ہر نماز کے لیے وضو ان پر لازم نہیں تھا اور نہ اس کا حکم ملا تھا اور آپ (صلی اللہ علیہ وآلہ وسلم) کو صرف حکم تھا اور اس میں آپ (صلی اللہ علیہ وآلہ وسلم) کا حکم ان سے مختلف تھا اس باب کی روایات کہ معنی کی تصحیح اسی طریق سے ہے اور اس سے ہر نماز کے لیے وضو لازم ہونے کے حکم کا اٹھ جانا بھی ثابت ہوگیا۔ بطور نظر و فکر کے اس کی وضاحت اس طرح ہے کہ وضو حدث سے طہارت کا کام دیتا ہے جب ہم احداث سے طہارتیں حاصل کرنے پر غور کرتے ہیں کہ ان کا حکم کیا ہے اور کونسی چیز طہارت کو توڑنے والی ہے تو ہم نے ایسی طہارتیں پائیں جو حدث سے لازم ہوتی ہیں ‘ ان کو دو قسم پر پایا۔ ایک ان میں سے غسل اور دوسرا وضو ہے۔ پس جس شخص نے جماع کیا یا اسے احتلام ہوا تو اس پر غسل لازم ہے اور جس نے پیشاب یاپاخانہ کیا تو اس پر وضو واجب ہے اور اس غسل واجب کو جس کا ہم نے ابھی تذکرہ کیا اوقات کا گزرنا نہیں توڑتا ‘ اس کو توڑنے والی چیز صرف حدث ہے۔ پس جب یہ چیز ثابت شدہ ہے کہ طہارت کا حکم جماع اور احتلام کی حالت میں ہے جیسا کہ ہم نے بیان کردیا تو غور و فکر کا تقاضا بھی یہی ہے کہ تمام طہارتوں کا حکم تمام احداث سے اسی طرح ہو کہ ان طہارتوں کو غسل کی طرح وقت کا گزرنا نہ توڑے ایک اور دلیل یہ ہے کہ ہم نے علماء کرام کو اس بات پر متفق پایا کہ مسافر ایک وضو سے تمام نمازیں پڑھے جب تک کہ حدث لاحق نہ ہو۔ مقیم کے بارے میں ان کو مختلف الرائے پایا۔ ہم نے غور کیا کہ احداث یہ چیزیں ہیں : جماع ‘ احتلام ‘ پیشاب و پاخانہ۔ ان میں سے جو چیز مقیم کو پیش آئے گی اس پر طہارت کو لازم کر دے گی۔ اس لیے کہ جب وہ مسافر تھا تو اس پر اسی طرح ہی لازم تھا اور اسی پر وہی طہارت لازم تھی جو مقیم ہونے کی حالت میں اس پر لازم ہوتی ‘ ہمیں ایک اور ایسی طہارت ملی جسے وقت کا نکلنا توڑ دیتا ہے اور اس میں مقیم و مسافر دونوں اس بات میں برابر ہیں کہ وقت کا نکلنا ان کی طہارت کو باطل کر دے۔ اگرچہ فی نفسہٖ وقت مقیم و مسافر کا الگ الگ ہو۔ پس جب یہ بات ثابت ہوگئی کہ جو ہم نے ذکر کیا وہ اسی طرح ہی ہے اور جو چیز مقیم کی طہارت کو توڑنے والی ہے وہی مسافر کی طہارت کو توڑنے والی ہے اور وقت کا نکل جانا جیسے مسافر کی طہارت کو نہیں توڑتا اسی طرح مقیم کی طہارت کو بھی نہیں توڑتا۔ قیاس و نظر تو ہمارے بیان کی تصدیق کرتے ہیں اور یہی امام ابوحنیفہ ‘ ابو یوسف و محمد (رح) کا مسلک ہے اور یہی بات جناب رسول اللہ (صلی اللہ علیہ وآلہ وسلم) کے بعد صحابہ (رض) اور تابعین (رح) کی جماعت نے کہی ہے۔
تخریج : ابو داؤد ١؍٦ مسلم ١؍١٢٨ بخاری ١؍٣٠٣ 
امام طحاوی (رح) کا ارشاد : 
 آپ (صلی اللہ علیہ وآلہ وسلم) کا ارشاد گرامی : لو لا ان اشق علی امتی لامرتھم بالسواک عند کل صلاۃ “ جس کو اوپر دس اسناد سے ذرا اختلاف کے ساتھ نقل کیا گیا ہے اس سے یہ بات ثابت ہوگئی کہ آپ (صلی اللہ علیہ وآلہ وسلم) نے امت پر لازم نہیں فرمایا (البتہ ترغیب دی) اور یہ حکم امت پر لازم بھی نہ تھا پھر اس کے منسوخ ہونے کی بات ان سے کیونکر منقول ہوتی۔
بلکہ وہ تو آپ (صلی اللہ علیہ وآلہ وسلم) کے لیے ہر نماز کے وضو کے بدلے لازم کیا گیا۔
 اور یہ اس بات کی دلیل بھی ہے کہ ہر نماز کے لیے وضو صحابہ کرام پر لازم نہ تھا اور نہ ہی ان کو اس کا حکم دیا گیا وہ حکم تو آپ کی ذات کے ساتھ خاص تھا اور اس سلسلہ میں آپ کا حکم دوسرے لوگوں سے مختلف تھا اگر باب کی روایات میں اس وجہ کو ملحوظ رکھا جائے تو روایات میں بآسانی موافقت ہوسکتی ہے اور اس سے یہ بات پورے طور پر ثابت ہوگئی کہ ہر نماز کے لیے وجوب وضو کا حکم آپ سے بھی اٹھا لیا گیا۔
طحاوی (رح) کی نظری دلیل نمبر ١: 
ہم نے غور کیا تو معلوم ہوا کہ وضو حدث سے طہارت کا نام ہے اب احداث سے طہارات کے احکام پر غور کیا تاکہ ہمیں طہارت کا حکم اور طہارت کو توڑنے والی چیزوں کا بخوبی علم ہوجائے غور کرنے پر طہارات کی کل دو قسمیں پائیں۔
نمبر ! غسل نمبر " وضو 
غسل : 
 ان لوگوں پر لازم ہے جو جنابت و جماع میں مبتلا ہوں۔
وضو : 
 ان لوگوں پر واجب ہے جو پیشاب و پائخانہ وغیرہ سے فارغ ہوں ذرا غور کرنے سے یہ بات معلوم ہو رہی ہے کہ غسل واجب جب کرلیا تو اوقات کا گزرنا اس کو نہیں توڑ سکتا غسل کے ٹوٹنے کی وہی صورتیں ہیں جو مذکور ہوئیں اور اوپر یہ بات ثابت ہوچکی کہ طہارت اکبر یعنی غسل کا حکم نجاست اکبر یعنی جماع و احتلام کے ساتھ خاص ہے اور فکرونظر کا فیصلہ یہی بنتا ہے کہ تمام احداث صغیرہ کبیرہ سے طہارت کا حکم اسی طرح ہونا چاہیے کہ وہ بھی غسل کی طرح فقط مرور زمانہ سے ٹوٹنے نہ پائے۔

دلیل ثانی ایک اور انداز سے توجہ فرمائیں : 
 اس بات پر سب کا اتفاق ہے کہ مسافر پانچوں نمازیں ایک ہی وضو سے پڑھ سکتا ہے جب تک کہ اس کا وضو نہ ٹوٹے البتہ مقیم کے متعلق اختلاف کیا گیا ہم نے دیکھا کہ مقیم کے متعلق حدث کو لازم کرنے والی چیزیں جماع ‘ احتلام ‘ پیشاب و پائخانہ ہے اور مسافر کے لیے بھی یہی ہیں مسافر پر بھی ان چیزوں سے طہارت لازم ہے جن سے مقیم کو لازم ہے البتہ طہارت کی ایک اور قسم مسح علی الخفین (موزوں پر مسح) ایسی ہے جس کے لیے وقت کا نکلنا بھی ناقض ہے اور اس میں مسافر و مقیم حکم میں یکساں ہیں اگرچہ مسافر و مقیم کے لیے وقت کی طوالت و قصر کا تو فرق ہے مگر طہارت کے ٹوٹنے میں قطعاً فرق نہیں۔
 پس یہ بات ثابت ہوگئی کہ جس چیز سے مسافر کی طہارت ٹوٹتی ہے اسی سے مقیم کی طہارت بھی ٹوٹتی ہے ان کے مابین نقض طہارت میں کوئی فرق نہیں مسافر کے لیے خروج وقت ناقض طہارت نہیں تو مقیم کے لیے پھر قیاس و نظر کے لحاظ سے کس طرح خروج وقت مبطل طہارت ہوگا۔ فتدبر۔ 
 یہی امام ابوحنیفہ (رح) اور ابو یوسف (رح) محمد (رح) کا قول ہے اور صحابہ وتابعین کی ایک کثیر جماعت کا یہی قول ہے جیسا مندرجہ روایات و آثار سے ظاہر ہے۔ 

" اور حضرت صفوان بن عسال (رض) فرماتے ہیں کہ " جب ہم سفر میں ہوتے تھے تو سرکار دو عالم (صلی اللہ علیہ وآلہ وسلم) ہمیں حکم دیتے تھے کہ تین دن اور تین راتوں تک (وضو کرنے کے وقت پاؤں کو (دھونے کے لئے) موزے نہ اتارے جائیں، نہ پاخانے کی وجہ سے نہ پیشاب کی وجہ سے نہ سونے کی وجہ سے البتہ جنابت کی وجہ سے (یعنی غسل واجب ہونے کی صورت میں نہانے کے لیے اتارے جائیں۔ " (جامع ترمذی، سنن نسائی)

تشریح 
 مطلب یہ ہے کہ سو کر اٹھنے یا پیشاب و پاخانے کے بعد وضو کرنے کی صورت میں اس مدت تک جو مسافر یا مقیم کے لیے ہے پاؤں کو دھونے کے لیے موزوں کو اتارنا نہیں چاہیے بلکہ موزوں پر مسح کرلیا جائے اور جنابت کی حالت میں یعنی جب غسل واجب ہوجائے تو نہانے کے لیے موزے اتارنے ضروری ہیں کیونکہ اس حالت میں موزوں پر مسح درست نہیں ہے۔ 

27054 ۔۔۔ حضرت عمر (رض) فرماتے ہیں : وضوان چیزوں سے واجب ہوتا ہے جو جسم سے باہر نکلتی ہیں جیسے پاخانہ پیشاب اور ہوا وغیرہ اور ان چیزوں سے نہیں ٹوٹتا جو بدن میں داخل ہوتی ہیں (رواہ سعید بن المنصور) 

864 ۔ حضرت علی بن ابوطالب (رض) بیان کرتے : میں نے نبی کریم (صلی اللہ علیہ وآلہ وسلم) سے اس پٹی کے بارے میں دریافت کیا جو کسی زخم وغیرہ پر باندھی جاتی ہے ایساشخص کس طرح وضو کرسکتا ہے ، اگر وہ جنبی ہوجائے تو وہ غسل کس طرح کرے گا ؟ تو نبی کریم (صلی اللہ علیہ وآلہ وسلم) نے فرمایا ایساشخص جنابت کی حالت میں پانی کے ذریعے اس پر مسح کرلے گا، پھر میں نے عرض کی ، اگر سردی ہو یا اپنی جان کا اندیشہ ہو ؟ تو نبی کریم (صلی اللہ علیہ وآلہ وسلم) نے فرمایا وہ اپنے پورے جسم پر مسح کرلے گا، پھر نبی کریم (صلی اللہ علیہ وآلہ وسلم) نے یہ آیت تلاوت کی :
'' تم اپنے آپ کو قتل نہ کرو بیشک اللہ تعالیٰ تمہارے بارے میں رحم کرنے والا ہے۔ (تو نبی کریم (صلی اللہ علیہ وآلہ وسلم) نے فرمایا) ایسے شخص کو جب اندیشہ ہو تو وہ تیمم کرلے۔ 

" اور حضرت ابوہریرہ (رض) فرماتے ہیں کہ سرکار دو عالم (صلی اللہ علیہ وآلہ وسلم) جب پاخانہ جاتے تو آپ کے لیے پیالہ یا چمڑے کی چھاگل میں پانی لاتا، رسول اللہ (صلی اللہ علیہ وآلہ وسلم) اس سے استنجاء کرتے پھر ہاتھ کو زمین پر رگڑتے پھر اس کے بعد میں (پانی کا) دوسرا برتن لاتا اور آپ (صلی اللہ علیہ وآلہ وسلم) وضو فرماتے۔ " (دارمی، سنن نسائی )

تشریح : 
 تور عرب میں پیتل یا پتھر کا ایک چھوٹا سا برتن پیالہ کی طرح ہوتا ہے، اس میں کھانا کھاتے ہیں اور بوقت ضرورت اس میں پانی بھر کر اس سے وضو بھی کرلیتے ہیں رکوۃ چمڑے کی چھاگل کو فرماتے ہیں جو پانی رکھنے کے کام آتا ہے۔
استنجاء سے فراغت کے بعد آپ (صلی اللہ علیہ وآلہ وسلم) زمین پر ہاتھ رگڑ کر اس لیے دھوتے تاکہ ہاتھ سے بدبو نکل جائے اور ہاتھ خوب پاک و صاف ہو چنانچہ پاخانہ سے آکر اس طرح سے ہاتھ دھونا سنت ہے۔
حضرت ابوہریرہ (رض) وضو کے لیے دوسرے برتن میں پانی اس لیے نہیں لاتے تھے کہ استنجے کے بقیہ پانی یا اس برتن سے وضو درست نہیں تھا بلکہ اس برتن میں چونکہ پانی صرف استنجے کی ضرورت کی مطابق ہی رہتا ہوگا اس لیے وضو کے لیے دوسرے برتن میں پانی لانے کی ضرورت ہوتی تھی، اس حدیث میں بعض علماء کرام نے یہ اخذ کیا ہے کہ اگر استنجے اور وضو کے پانی کے لیے الگ الگ برتن ہوں تو یہ مستحب ہے۔ 

 (١٤٧٣) حضرت مالک بن انس فرماتے ہیں کہ صرف اس چیز سے وضو ٹوٹتا ہے جو سبیلین سے نکلے یعنی پیشاب اور پاخانہ۔ 

19917 ۔۔۔ ضحک (ہنسنا) نماز کو توڑ دیتا ہے لیکن وضو کو نہیں توڑتا ۔ (السنن للدار قطنی عن جابر (رض)) فائدہ :۔۔۔ ایسی ہنسی ہنسنا کہ آواز آس پاس کے لوگوں کو پہنچ جائے اس سے نماز اور وضو دونوں ٹوٹ جاتے ہیں۔ اگر ہنسی کی آواز صرف خود کو پہنچے دوسروں کو نہ پہنچے تو اس سے نماز ٹوٹ جاتی ہے جب کہ وٖضو باقی رہتا ہے اور اگر صرف تبسم کیا جائے جس میں آواز نہ ہو تو اس تبسم (مسکراہٹ) سے نماز جاتی ہے اور نہ وضوء۔ 

" اور حضرت عمار ابن یاسر (رض) یہ بیان کرتے ہیں کہ (" ایک دفعہ) چند صحابہ کرام رسول اللہ (صلی اللہ علیہ وآلہ وسلم) کے ہمراہ تھے اور (پانی نہ ملنے کی وجہ سے) فجر کی نماز کے لیے انہوں نے پاک مٹی سے (اس طرح) تیمم کیا (کہ پہلے) اپنے ہاتھوں کو مٹی پر مار کر اپنے چہروں پر پھیرا اور دوسری مرتبہ اپنے ہاتھوں کو مٹی پر مار کر اپنے پورے ہاتھوں پر یعنی کندھوں تک اور بغلوں کے اندر تک مسح کیا ہاتھوں کے اندر کی طرف سے۔ " (سنن ابوداؤد)

تشریح 
 حدیث کے آخری جملے من بطون ایدیھم میں لفظ من ابتدا کے لیے ہے یعنی انہوں نے پہلے ہاتھوں کے اندر کے رخ پر ہاتھ پھیرے نہ کہ ہاتھوں کے اوپر کے رخ پر، جیسا کہ فقہاء نے لکھا ہے کہ پہلے ہاتھوں کے اوپر کے رخ پر مسح کرنا مستحب ہے۔ یا پھر اس کے معنی یہ ہوئے کہ " انہوں نے " ہتھیلیوں سے تیمم کرنا شروع کیا۔ " یہی معنی زیادہ مناسب ہیں۔
صحابہ کرام رضوان اللہ علیہم اجمعین نے ہاتھوں پر بغلوں اور مونڈھوں تک مسح کیا اس کی وجہ یہ تھی کہ انہوں نے خیال کیا کہ آیت تیمم میں " ید " یعنی ہاتھ کا لفط مذکور ہے جو مطلق ہے۔ اس آیت سے اور اس لفظ سے یہ بصراحت معلوم نہیں ہوتا کہ ہاتھوں پر مسح کہاں تک کیا جائے لہٰذا اس اطلاق کا تقاضا یہ ہے کہ " ید " یعنی ہاتھ جو انگلیوں سے لے کر بغل اور کندھوں تک کے حصے کے لیے بولا جاتا ہے اس پورے حصے پر مسح کیا جائے اس لیے صحابہ کرام رضوان اللہ علیہم اجمعین نے ہاتھ کے پورے حصے پر مسح کر ڈالا۔ اب جہاں تک مسئلے کا تعلق ہے تو یہ کہا جائے گا کہ یہ صحابہ کرام رضوان اللہ علیہم اجمعین کا اپنا اجتہاد تھا کیونکہ جمہور علماء کرام رضوان اللہ علیہم اجمعین نے تیمم میں ہاتھوں پر کہینوں تک مسح کرنے کا جو حکم دیا ہے اس کی وجہ یہ ہے کہ تیمم وضو کا قائم مقام ہے اور وضو کے بارے میں قرآن نے صراحت کے ساتھ بتادیا کہ ہاتھوں کو کہینوں تک دھونا فرض ہے لہٰذا جس طرح اصل یعنی وضو میں ہاتھوں کو کہنیوں تک دھویا جاتا ہے۔ وضو کے قائم مقام یعنی تیمم میں بھی ہاتھوں پر مسح وہیں تک کیا جانا چاہیے۔
پھر اس سے پہلے تیمم کے بارے میں کچھ احکام نقل کئے گئے تھے اس حدیث پر چونکہ باب ختم ہو رہا ہے اس لیے مناسب ہے کہ تیمم کے چھ دوسرے احکام و مسائل جو پہلے نقل نہیں کئے گئے ذکر کر دئیے جائیں۔
پہلے بتایا جا چکا ہے کہ تیمم ان چیزوں سے کرنا چاہیے جو زمین کی جنس سے ہوں چنانچہ مٹی، ریت، چونا، قلعی، سرمہ، ہڑتال اور پتھر سے تیمم کیا جاسکتا ہے، اسی طرح موتی اور مونگے کے علاوہ تمام جو اہرت سے بھی تیمم کیا جاسکتا ہے۔
یہ بھی سمجھ لیجئے کہ تیمم کرنے کے لیے ان چیزوں پر جو زمین کی جنس سے ہوں غبار ہونا ضروری نہیں ہے۔ یعنی ان مذکور چیزوں پر غبار نہ ہونے کی صورت میں بھی ان سے تیمم کیا جاسکتا ہے۔ ہاں اگر ایسی چیز کے ذریعے تیمم کیا جائے جو زمین کی جنس سے نہ ہو تو اس پر غبار ہونا ضروری ہے، غبار نہ ہونے کی شکل میں اس کے ذریعے کیا گیا تیمم جائز نہ ہوگا، مثلاً کسی لکڑی، کپڑے یا سونے اور چاندی وغیرہ پر غبار ہو تو اس سے تیمم جائز ہے۔
تیمم کے جواز کے لیے چار شرائط ہیں۔ (١) پانی کے استعمال سے حقیقۃً یا حکمًا عاجز ہونا۔ (٢) جس چیز سے تیمم کیا جائے اس کا پاک ہونا۔ (٣) استیعاب یعنی اعضاء تیمم کے ہر ہر حصے پر اس طرح ہاتھ پھیرنا کہ کوئی جگہ مسح سے باقی نہ رہ جائے۔ (٤) نیت اس کے بارے میں پہلے بتایا جا چکا ہے کہ نماز اسی تیمم سے صحیح ہوگی جس میں حدث سے طہارت کی نیت کی جائے۔ اس سلسلے میں (حدث یا جنابت کی تعیین شرط نہیں ہے) یا اس سے عبادت مقصودہ کی نیت کی جائے جو بغر طہارت کے صحیح نہیں ہوتی۔ چنانچہ اگر کافر اسلام قبول کرنے کے لیے تیمم کرے یا کوئی آدمی مسجد میں جانے کے لیے تیمم اور پھر یہ چاہیے کہ اسی تیمم سے نماز پڑھ لے تو نماز اس تیمم سے جائز نہ ہوگی۔ جنبی، محدث، حائضہ اور نفاس والی عورت سب کے لیے تیمم کا ایک ہی طریقہ ہے جو پہلے ذکر کیا جا چکا ہے۔ 

 ہشام بن عمار، صدقہ بن خالد، عتبہ بن ابی حکیم، طلحہ بن نافع ابوسفیان، حضرت ابوایوب انصاری، جابر بن عبداللہ، انس بن مالک (رض) فرماتے ہیں کہ آیت (رِجَالٌ يُّحِبُّوْنَ اَنْ يَّتَطَهَّرُوْا وَاللّٰهُ يُحِبُّ الْمُطَّهِّرِيْنَ ١٠٨) 9 ۔ التوبہ : 108) اتری تو رسول اللہ (صلی اللہ علیہ وآلہ وسلم) نے فرمایا اے گروہ انصار اللہ تعالیٰ نے طہارت کی وجہ سے تمہاری تعریف فرمائی ہے تو تم طہارت کیسے حاصل کرتے ہو۔ انہوں نے عرض کیا نماز کے لیے وضو کرتے ہیں۔ جنابت ہوجائے تو غسل کرتے ہیں اور پانی سے استنجا کرتے ہیں۔ فرمایا بس یہی وجہ ہے تم اس کو تھامے رکھو۔ 

 اس باب میں ان چیزوں کا ذکر کیا جا رہا ہے جو وضو کو توڑتی ہیں چنانچہ حضرت امام اعظم ابوحنیفہ (رح) کے مسلک کے مطابق ان چیزوں سے وضو ٹوٹ جاتا ہے۔
(١) پاخانہ اور پیشاب کے راستہ سے نکلنے والی ہر چیز سے وضو ٹوٹ جاتا ہے جیسے پاخانہ، پیشاب اور ریاح وغیر مگر جو ہوا مرد یا عورت کے آگے کے سرے سے نکلتی ہے اس سے وضو نہیں ٹوٹتا۔
(٢) اس چیز سے وضو ٹوٹ جاتا ہے جو نجس ہو (جیسے خون اور پیپ وغیرہ) اور بدن میں خود بخود نکل کر اس حصہ تک پہنچ جائے جس کو غسل یا وضو میں دھونا لازم ہو، یعنی اگر ناک کے بانسے اور آنکھ کے اندر رہے تو اس سے وضو نہیں ٹوٹے گا کیونکہ ان کا دھونا لازم نہیں ہے۔
(٣) قے کرنے سے وضو ٹوٹ جاتا ہے منہ بھر قے کرنے میں خواہ اناج نکلے، پانی نکلے، جما ہوا خون یعنی سودا نکلے ان سے وضو ٹوٹ جاتا ہے، اگر بلغم نکلے تو وضو نہیں ٹوٹتا، اگر پتلے خون یا پیپ کی قے ہو تو اس میں منہ بھرنے کی شرط نہیں بلکہ تھوک کے برابر ہو یا تھوک پر غالب ہوجائے تب بھی وضو ٹوٹ جائے گا اور اگر کم ہوگا تو نہیں ٹوٹے گا اگر ایک ہی متلی میں تھوڑی قے اتنی مقدار میں ہوئی کہ اگر اسے جمع کیا جائے تو منہ بھر جائے تو اس سے وضو جاتا ہے جس چیز سے وضو نہیں ٹوٹتا ہے وہ نجس نہیں ہوتی مثلاً تھوڑی سے قے کی یا بدن سے خون اس طرح نکلا کہ وہ جسم پر بہا نہیں تو یہ ناپاک نہیں ہے۔
(٤) وضوٹوٹ جاتا ہے دیوانہ ہونے سے۔
(٥) نشے سے۔
(٦) بےہوش ہوجانے سے۔
(٧) اور بالغ کے قہق ہے سے اس نماز میں جو رکوع و سجود والی ہو۔
(٨) مباشرت فاحشہ سے وضو ٹوٹ جاتا ہے، مباشرت فاحشہ اسے فرماتے ہیں کہ انتشار اور جنسی ہیجان کے ساتھ مرد کا ستر عورت کے ستر سے اور عورت کا ستر مرد کے ستر سے مل جائے یا دو عورتوں یا مردوں کے ستر مل جائیں۔
(٩) لیٹ کر اپنے بدن پر یا دیوار وغیرہ پر تکیہ لگا کر سونے سے وضو ٹوٹ جاتا ہے لیکن یہ سونا اس طرح ہو کہ اگر تکیہ کی وہ چیز جس پر ٹیک لگا کر سویا ہوا ہے ہٹالی جائے تو گرپڑے۔
(١٠) اگر اس طرح سو جائے کہ مقعد زمین سے اٹھ جائے یعنی پہلو پر یا کو لھوں پر یا چت یا منہ کے بل، یا کو لھے کو دیوار سے لگا کر یا پیٹ پاؤں پر لگا کر جھکا ہوا سو جائے تو وضو ٹوٹ جاتا ہے اور اگر کھڑا کھڑا سو جائے یا رکوع اور سجدہ کی حالت میں سو جائے تو وضو نہیں ٹوٹتا مگر شرط یہ ہے کہ رکوع و سجود ہیت مسنونہ پر ہوں، اگر زخم میں کیڑے نکلیں یا گوشت کٹ کر گرجائے تو وضو نہیں ٹوٹتا۔
(١١) اگر جونک لگائی جائے اور وہ خون پی کی کر بھر گئی یا بڑی چیچری نے پیٹ بھر خون پیا تو وضو ٹوٹ جاتا ہے اور اگر ایسا نہیں ہے تو وضو نہیں ٹوٹتا۔
(١٢) اگر کسی کی آنکھ دکھنے آتی ہے اور آنسو نکلتے ہیں تو وضو ٹوٹ جاتا ہے، اس سلسلہ میں اکثر لوگ غافل ہیں اس کا خیال نہیں کرتے اس لیے اس کا خیال رکھنا چاہیے ہاں اگر کوئی آدمی ایسا ہے جس کی آنکھیں ہمیشہ جاری رہتی ہیں تو وہ صاحب عذر ہوجاتا ہے۔
(١٣) اگر کان دکھتا ہے اور اس سے پیپ یا کچھ لہو نکلے تو وضو ٹوٹ جاتا ہے ان میں سے دو چیزیں یعنی پیشاب اور پاخانہ کے راستہ سے نکلنے والی چیزوں اور نیند پر تمام علماء کا اتفاق ہے کہ یہ چیزیں ناقض وضو ہیں باقی چیزیں مختلف فیہ ہیں۔ 

In [37]:
print(query_words)

['وضو', 'وضو', 'وضو', 'کس', 'کس', 'کس', 'طرح', 'طرح', 'طرح', 'کریں', 'کریں', 'کریں', 'غسل', 'جنابت', 'تیمم', 'مسواک', 'استنجاء', 'کب', 'مانند', 'کرسکیں', 'کرلیں', 'کیجیے', 'کیجئے', 'کرو']
